In [1]:
import pandas as pd
import numpy as np
from age_gap_automation import Movie
from pyvis.options import EdgeOptions
from pyvis.network import Network

import sys
sys.path.append("../..")
import bechdelai.data.tmdb as tmdb

hp4 = pd.read_csv('hp4.csv')

hp4_data = Movie("Harry Potter and the Goblet of Fire",2005)
cast = hp4_data.cast

# Plot Characters relationships

In [23]:
# Define color by relationship type
relationship_type = {'kisses':'love', 
                      'sleeps with':'love', 
                      'goes on a date with':'love', 
                      'has sex with':'love', 
                      'marries':'love', 
                      'is in love with':'love',
                      'is in couple with':'love',
                      'is the father of':'family', 
                      'is the mother of':'family',
                      'is a friend of':'friend', 
                      'is in the family of':'family', 
                      'is the enemy of':'enemy'}
relationship_color = {'love':'#fca311', 
                      'family':'#cce03d', 
                      'friend':'#35c4d7', 
                      'enemy':'#000000'}

In [4]:
# get edges
hp4 = hp4.sort_values('score',ascending=False)
hp4 = hp4[hp4.score>0.4]

all_stars = np.unique(list(hp4.star1.values)+list(hp4.star2.values))

In [35]:
g = Network()

# add nodes
for star in all_stars:
    star_data = cast[cast.name==star]
    character = star_data['character'].iloc[0]
    image_path = tmdb.get_person_image_from_id(star_data.id.iloc[0])
    image = 'https://image.tmdb.org/t/p/original'+image_path["profiles"][0]["file_path"]
    g.add_node(star, 
               label = character,
               title = star,
               shape='circularImage', image =image,
               borderWidth=0,
               size = 30,
               labelHighlightBold=True)

# add edges
for i,row in hp4.iterrows():
    row_relation_type = relationship_type[row.question]
    title = "At movie release\n"+\
            '{} was {} years old \n'.format(row.star1,cast[cast.name==row.star1]['age_at_release'].iloc[0])+\
            '{} was {} years old \n'.format(row.star2,cast[cast.name==row.star2]['age_at_release'].iloc[0])
    
    
#     if row_relation_type=='love': # is romantic relationship
#         title = age_gap
    g.add_edge(row.star1,row.star2, 
               value=100*row.score,
               color=relationship_color[row_relation_type],
               title = title)

In [37]:
g.show("hp4_chars_relationships_circular.html")